In [109]:
#pip install tensorflow-gpu==1.15

ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==1.15 (from versions: none)
ERROR: No matching distribution found for tensorflow-gpu==1.15
Note: you may need to restart the kernel to use updated packages.


In [112]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd
import re
tf.__version__

'2.0.0'

In [111]:
lines =open('movie_lines.txt',encoding="utf-8",errors='ignore').read().split('\n')
conversations =open('movie_conversations.txt',encoding="utf-8",errors='ignore').read().split('\n')

In [4]:
#dictionary with key:movie sentence id, value:sentence
id2line={}
for line in lines:
    _line = line.split(" +++$+++")
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [5]:
#list with dic key:movie sentence id, and each list with sen_ids consist a conversation.
## trail
print(conversations[0])
# _c = conversations[0].split(" +++$+++")[-1][2:-1].replace('"',"").replace("'","")
# c = _c.split(", ")
# c

conversation_ids=[]
for conversation in conversations:
    _conversation = conversation.split(" +++$+++")[-1][2:-1].replace('"',"").replace("'","")
    conversation_ids.append(_conversation.split(", ")) #每一段对话是一行列表。
    
#print(conversation_ids[0])
if 'L194' in conversation_ids: #'L3515'
    print("T")
else:
    print("F") #应该是f，因为ids的基本单元是一串列表。

u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']
F


In [6]:
#test
id2line[conversation_ids[0][2]] #first line of con_ds have 4 sentences.
id2line[conversation_ids[0][3]]

list = [1,2]
len(list)
range(len(list)-1) #this loop have to be in the range(),rather than the len().


range(0, 1)

In [7]:
#turn the each list of conversation into Q & A.
Q =[]
A =[]
for conversation in conversation_ids:  #each line,each list
    for i in range(len(conversation)-1): #each index of each sentence. 依次遍历每一段coversation的长度（从0，1，2...到长度）
#         print(conversation_ids[i])
        Q.append(id2line[conversation[i]]) 
        A.append(id2line[conversation[i+1]]) #上一句的答，可以是下一句的问。 range最右边界取不到，所以不会超出边界。
# Q
A

[" Well, I thought we'd start with pronunciation, if that's okay with you.",
 ' Not the hacking and gagging and spitting part.  Please.',
 " Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?",
 ' Forget it.',
 ' Cameron.',
 " The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
 ' Seems like she could get a date easy enough...',
 ' Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something.',
 " That's a shame.",
 ' Let me see what I can do.',
 " Right.  See?  You're ready for the quiz.",
 " I don't want to know how to say that though.  I want to know useful things. Like where the good stores are.  How much does champagne cost?  Stuff like Chat.  I have never in my life had to point out my head to someone.",
 " That's because it's such a nice one.",
 ' Forget French.',
 " Well, there's someone I think might be --",


In [8]:
#data cleaning1 eg."i'm" == "i am"

## test
# s = "i'm a boy."
# sc = re.sub(r"i'm","i am",s)
# sc

def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"he's","he is",text)
    text = re.sub(r"she’s","she is",text)
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"\'d"," would",text)
    text = re.sub(r"\'ll'"," will",text)
    text = re.sub(r"won't","will not",text)
    text = re.sub(r"can't","can not",text)
    text = re.sub(r"  "," ",text)
    text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,]","",text)
    return text


clean_q=[]
clean_a=[]
for q in Q:  
    clean_q.append(clean_text(q))
for a in A:  
    clean_a.append(clean_text(q))

# add "eos" tag  
for i in range(len(clean_q)):#range最右边取不到，不会超出边界。
    clean_q[i] +=" <EOS>" #每一句末尾加eos
for i in range(len(clean_a)):#range最右边取不到，不会超出边界。
    clean_a[i] +=" <EOS>"

In [9]:
qw_and_c

NameError: name 'qw_and_c' is not defined

In [13]:
#data cleaning2-del low freq words
## test
# clean_q[0]
# for word in clean_q[0].split(" "):
#     print(word)

# create a word count dic.
qw_and_c_ = {}
aw_and_c_ = {}
invalid = [" ",""]
for q in clean_q:
    for word in q.split(" "):
        if word not in invalid:
            if word not in qw_and_c_:
                qw_and_c_[word] = 1
            else:
                qw_and_c_[word] += 1
for a in clean_a:
    for word in a.split(" "):
        if word not in invalid:
            if word not in aw_and_c_:
                aw_and_c_[word] = 1
            else:
                aw_and_c_[word] += 1
# qw_and_c["we"]

#create a new dic with word_counts > 20.
lowest = 20
qw_and_c = {}
aw_and_c = {}
zero = 0 
ints=[]
#type(q_and_c.items())
for q,c in qw_and_c_.items():
    if c >= lowest:
        qw_and_c[q] = c
for a,c in aw_and_c_.items():
    if c >= lowest:
        aw_and_c[a] = c

In [14]:
# encode:word to int; decode:int to word.
# so, I have to create a reverse dic with count as key, word as value.
c_and_qw ={c:q for q,c in qw_and_c.items()}
c_and_aw ={c:a for a,c in aw_and_c.items()}

In [15]:
clean_q[0].split(" ")


['',
 'can',
 'we',
 'make',
 'this',
 'quick',
 'roxanne',
 'korrine',
 'and',
 'andrew',
 'barrett',
 'are',
 'having',
 'an',
 'incredibly',
 'horrendous',
 'public',
 'break',
 'up',
 'on',
 'the',
 'quad',
 'again',
 '<EOS>']

In [16]:
#turn clean_q(sentences) into int(use counts to represent words) #duplicate
## check
# len(clean_q)
# len(qw_and_c.items())
# len(c_and_qw) #为啥长度变短了？？

intq = []
for q in clean_q:#each sentence
    ints=[]
    for word in q.split(" "):
        if word not in invalid:
            if word in qw_and_c:
                ints.append(qw_and_c[word])
    intq.append(ints)

inta = []
for a in clean_a:#each sentence
    ints=[]
    for word in a.split(" "):
        if word not in invalid:
            if word in aw_and_c:
                ints.append(aw_and_c[word])
    inta.append(ints)

In [44]:
#长短句处理
sorted_clean_q = []
sorted_clean_a = []
for length in range(1,25+1):
    for i_wlist in enumerate(intq):
        #print(len(i_wlist))
       # print(len(i_wlist[1])) #length of each int sentence
        if len(i_wlist[1]) == length: #长度从1-25依次排序
            sorted_clean_q.append(intq[i_wlist[0]])
            #sorted_clean_a.append(i_wlist[0])
sorted_clean_q

[[221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],
 [221616],

In [ ]:
#seqtoseq:编码encode-char2int(lstm)，解码decode(int2char)

In [98]:
def decode_training_set(encoder_state,decoder_cell,decoder_embedded_input,sequence_length,decoding_scope,output_function,
                       keep_prob,batch_size):
    attention_states = tf.zeros([batch_size,1,decoder_cell.output_size])
    attention_keys,attention_values,attention_score_function,attention_construct_function= tf.seq2seq.prepare_attention(attention_states,attention_option= "bahdanau" ,num_units=decoder_cell.output_size)
    training_decoder_function = tf.seq2seq.attention_decoeder_fn_train(
    encoder_state[0],attention_keys,attention_values,attention_construct_function,
    name="attn_dec_train")
    decoder_output,decoder_final_state,
    decoder_final_context_state = tf.seq2seq.dynamic_rnn_decoder(decoder_cell,
                                                                        training_decoder_function,
                                                                        decoder_embedded_input,
                                                                        sequence_length,
                                                                        scope = decoding_scope)
    decoder_output_dropout = tf.nn.dropout(decoder_otput,keep_prob)
    return output_function(decoder_output_dropout)



def decode_test_set(encoder_state,decoder_cell,decoder_embedded_matrix,sos_id,eos_id,maximum_length,num_words,decoding_scope,output_function,
                       keep_prob,batch_size):
    attention_states = tf.zeros([batch_size,1,decoder_cell.output_size])
    attention_keys,attention_values,attention_score_function,attention_construct_function= tf.seq2seq.prepare_attention(attention_states,attention_option= "bahdanau" ,num_units=decoder_cell.output_size)
    test_decoder_function = tf.seq2seq.attention_decoeder_fn_train(
    encoder_state[0],attention_keys,attention_values,attention_score_function,attention_construct_function,
        decoder_embeddings_matrix,sos_id,eos_id,maximum_length,num_unitsm_words,
    name="attn_dec_inf")
    test_predictions,decoder_final_state,decoder_final_context_state = tf.seq2seq.dynamic_rnn_decoder(decoder_cell,test_decoder_function,scope = decoding_scope)
    decoder_output_dropout = tf.nn.dropout(decoder_otput,keep_prob)
    return test_predictions

In [99]:
def encoder_rnn(rnn_inputs,rnn_size,keep_prob,num_layers,sequence_length):
    lstm = tf.rnn.BasicLSTMCell(rnn_size)
    lstm_dropout = tf.rnn.DropoutWrapper(lstm,input_keep_prob = keep_prob)
    encoder_cell = tf.rnn.MultiRNNCell([lstm_dropout]*num_layers )
    
    encoder_output,encoder_state =  tf.nn.bidirectional_dynamic_rnn(
                                    cell_fw = encoder_cell,
                                    cell_bw = encoder_cell,
                                    sequence_length = sequence_length,
                                    inputs = rnn_inputs,
                                    dtype = tf.float32)
    return encoder_state
        
def decoder_rnn(decoder_embedded_input,
                decoder_embeddings_matrix,
                encoder_state,
                num_words,
                sequence_length,
                rnn_size,keep_prob,num_layers,word2int,batch_size):
    with tf.variable_scope("decoding") as decoding_scope:
        lstm = tf.rnn.BasicLSTMCell(rnn_size)
        lstm_dropout = tf.rnn.DropoutWrapper(lstm,input_keep_prob = keep_prob)
        decoder_cell = tf.rnn.MultiRNNCell([lstm_dropout]*num_layers )
        
        weights = tf.truncated_normal_initializer(stddev=0.1)
        biases = tf.zeros_intializer()
        output_function = lambda x : tf.layers.fully_connected(x,
                                                               num_words,None,
                                                               scope = decoding_scope,
                                                               weights_initializer = weights,
                                                               biases_initializer = biases)
        training_predicitons = decode_training_set(encoder_state,
                                                   decoder_cell,
                                                   decoder_embedded_input,
                                                   sequence_length,
                                                   decoding_scope,output_function,
                                                   keep_prob,batch_size)
        decoding_scope.reuse_variables()
        test_predictions = decode_test_set(encoder_state,
                                           decoder_cell,
                                           decoder_embedded_matrix,
                                           word2int["<SOS>"],
                                           word2int["<EOS>"],
                                           maximum_length,
                                           num_words,
                                           decoding_scope,output_function,
                                           keep_prob,batch_size)
    return training_predicitons,test_predictions
           

In [124]:
def seq2seq_model( inputs,target,keep_prob,batch_size,sequence_length,answers_num_words,
                 questions_num_words,encoder_embedding_size,decoder_embedding_size,rnn_size,num_layers,
                 questionswords2int):
    encoder_embedded_input = tf.embed_sequence(inputs,answers_num_words+1,encoder_embedding_size,
                                                       #      initializer = tf.random_uniform_initializer(0,1))
    
    encoder_state = encoder_rnn(encoder_embedded_input,rnn_size,num_layers,keep_prob,sequence_length)
   
    #preprocessed_targets = preprocess_targets(targets, qw_and_c,batch_size  )
    decoder_embedding_matrix = tf.Variable(tf.random_uniform([ questions_num_words +1, decoder_embedding_size],0  ) )
    decoder_embedded_input = tf.nn.embedding_lookup( decoder_embedding_matrix ,preprocessed_targets)
    
    training_predictions,test_predictions = decoder_rnn(decoder_embedded_input,
                                                        decoder_embeddings_matrix,
                                                        encoder_state,
                                                        questions_num_words,
                                                        sequence_length,
                                                        rnn_size,keep_prob,num_layers,qw_and_c,batch_size)

SyntaxError: invalid syntax (<ipython-input-124-d6323f491f4d>, line 10)

In [114]:
epochs = 100
batch_size = 64
rnn_size = 512
num_layers = 3
encoding_embedding_size =512
decoding_embedding_size =512
sequence_length =25
learning_rate_decay =0.09 #0.9?
min_learning_rate =0.0001


tf.reset_default_graph()
session = tf.InteractiveSession()

# def model_inputs():
#     inputs =[None,None]
#     targets = [None,None]
lr =0.01
keep_prob = 0.5 #according to essay
#     return inputs,targets,lr,keep_prob
# inputs,targets,lr,keep_prob = model_inputs()
#input_shape = tf.shape(input)

/Applications/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [119]:
training_predictions, test_predictions = seq2seq_model(
    inputs,targets,keep_prob,batch_size,sequence_length,len(qw_and_c),
    len(aw_and_c),encoding_embedding_size,decoding_embedding_size,rnn_size,num_layers,
    qw_and_c)


NameError: name 'encoder_embedded_input' is not defined